In [1]:
import pandas as pd
import os
import numpy as np
from glob import glob
home=os.path.expanduser("~")

# input rawdata

In [ ]:
year=2017
exls=glob(os.path.join(home,'station2grid','datasets','rawdata','epa','%s*'%(year-1911),'*','*xls'))
exl=exls[2]
exl

In [ ]:
oneSite=pd.read_excel(exl,encoding='big5')
oneSite.head()

# parse data

In [ ]:
station=oneSite.iloc[0,1]
station

In [ ]:
i=0
row=oneSite.iloc[i:i+1,:]
row

In [ ]:
date=row.iloc[0,0]
date

In [ ]:
feature=row.iloc[0,2]
feature

In [ ]:
vals=row.iloc[0,3:].values
vals

In [ ]:
block=pd.DataFrame()
block['dt']=pd.date_range(start=date, periods=24,freq='H')
block['value']=vals
block['station']=station
block['feature']=feature
block=block[['dt','station','feature','value']]
block.head()

In [19]:
def row2block(row,includeCols,excludeSites):
    feature=row.iloc[0,2]
    station=row.iloc[0,1]
    if feature not in includeCols: return 
    if station in excludeSites: return 
    
    date=row.iloc[0,0]
    vals=row.iloc[0,3:].values
    
    block=pd.DataFrame()
    block['dt']=pd.date_range(start=date, periods=24,freq='H')
    block['value']=vals
    block['station']=station
    block['feature']=feature
    block=block[['dt','station','feature','value']]
    return block

# block=row2block(row,['WS_HR'])
# block.head()

# script

In [29]:
def process_value(x):
    if x=='NR': return 0
    try: return float(x)
    except: return np.nan
    
def get_oneSite(exl):
    includeCols=['PM2.5','PM10','AMB_TEMP','RH']
    excludeSites=['富貴角','馬公','馬祖','金門']
    oneSite=pd.read_excel(exl,encoding='big5')
    
    oneSite_=pd.DataFrame()
    for i in range(len(oneSite))[:]:
        row=oneSite.iloc[i:i+1,:]
        block=row2block(row,includeCols,excludeSites)
        oneSite_=oneSite_.append(block)
    return oneSite_
    
def get_oneYear(year):
    exls=glob(os.path.join(home,'station2grid','datasets','rawdata','epa','%s*'%(year-1911),'*','*xls'))
    oneYear=pd.DataFrame()
    for i,exl in enumerate(exls[:2]):
#         print(i,len(exls))
        oneSite=get_oneSite(exl)
        oneYear=oneYear.append(oneSite)
        oneYear['value']=oneYear.value.apply(process_value)
    return oneYear


In [36]:
oneYear=get_oneYear(2014)
oneYear.head()

,dt,station,feature,value
0,2014-01-01 00:00:00,臺東,AMB_TEMP,18.0
1,2014-01-01 01:00:00,臺東,AMB_TEMP,18.0
2,2014-01-01 02:00:00,臺東,AMB_TEMP,18.0
3,2014-01-01 03:00:00,臺東,AMB_TEMP,18.0
4,2014-01-01 04:00:00,臺東,AMB_TEMP,18.0


In [41]:
oneYear=oneYear.dropna()

In [46]:

oneYear[oneYear.value.isna()]

,dt,station,feature,value


In [47]:
oneYear.iloc[0,3]

18.0